In [1]:
import deepchem as dc # requires python3<=3.11, numpy<2.0.0, tensorflow==2.15.0
dc.__version__

Skipped loading some Pytorch utilities, missing a dependency. No module named 'torch'
No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!


This module requires PyTorch to be installed.


2024-06-17 15:41:44.333448: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-17 15:41:44.356031: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-17 15:41:44.356054: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-17 15:41:44.356643: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-17 15:41:44.360608: I tensorflow/core/platform/cpu_feature_guar

Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading some PyTorch models, missing a dependency. No module named 'torch'
No module named 'torch'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'torch'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


'2.8.1.dev'

In [2]:
dataset_file = 'datasets/train_set_no_oversampling.csv'
task = ['Biodegradation']
featurizer_func = dc.feat.WeaveFeaturizer()

loader = dc.data.CSVLoader(tasks=task, feature_field='smiles', featurizer=featurizer_func)
dataset = loader.create_dataset(dataset_file)
transformer = dc.trans.BalancingTransformer(dataset=dataset)
balanced_dataset = transformer.transform(dataset)

In [9]:
import tensorflow as tf

In [16]:
model = dc.models.WeaveModel(n_tasks=1, mode='classification', dropout=0.25, batch_normalize=False, fully_connected_layer_sizes=[2000,100], final_conv_activation_fn=tf.nn.tanh)

/home/matcha/Code/deepchem_net/venv/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [17]:
model.fit(balanced_dataset, nb_epoch=200)

0.2883803939819336

In [19]:
test_set = 'datasets/test_set.csv'
featurizer_func = dc.feat.WeaveFeaturizer()
task = ['Biodegradation']

loader = dc.data.CSVLoader(feature_field='smiles', featurizer=featurizer_func, tasks=task)
test_dataset = loader.create_dataset(test_set)

transformer = dc.trans.BalancingTransformer(dataset=test_dataset)
balanced_test_dataset = transformer.transform(test_dataset)

model.evaluate(balanced_test_dataset, [dc.metrics.Metric(dc.metrics.roc_auc_score), dc.metrics.Metric(dc.metrics.matthews_corrcoef), dc.metrics.Metric(dc.metrics.recall_score) ])

{'roc_auc_score': 0.9030251141552511,
 'matthews_corrcoef': 0.6195497700045345,
 'recall_score': 0.8310502283105022}